In [ ]:
!pip install transformers

In [ ]:
!pip install torch

#### Downlaoding dataset

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1upXO4eN0dP8qKb0LL9hxS-J13BpHl7Dk' -O 'tweets_combined.csv'
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1nPww8vxj9NDO1ddVce-PM3F4y3bD59Pl' -O 'price_combined.csv'
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1ka75BqQ65SPxD-zr8goJIzoKmLLw09Uw' -r -A 'uc*' -e robots=off -nd

--2021-12-14 03:46:47--  https://drive.google.com/uc?export=download&id=1upXO4eN0dP8qKb0LL9hxS-J13BpHl7Dk
Resolving drive.google.com (drive.google.com)... 142.251.8.138, 142.251.8.102, 142.251.8.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.8.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/o9g8jjfkqkrej8d7447f9ea7ctgvnc7r/1639453575000/14724390508833084599/*/1upXO4eN0dP8qKb0LL9hxS-J13BpHl7Dk?e=download [following]
--2021-12-14 03:46:51--  https://doc-10-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/o9g8jjfkqkrej8d7447f9ea7ctgvnc7r/1639453575000/14724390508833084599/*/1upXO4eN0dP8qKb0LL9hxS-J13BpHl7Dk?e=download
Resolving doc-10-18-docs.googleusercontent.com (doc-10-18-docs.googleusercontent.com)... 142.251.8.132, 2404:6800:4008:c15::84
Connecting to doc-10-18-docs.googleusercontent.com (doc-10-18-d

In [ ]:
!ls

 price_combined.csv
 sample_data
 tweets_combined.csv
 twitter_sentiment_dataset.csv
 twitter_test.csv
 twitter_train.csv
'uc?export=download&confirm=CGXa&id=1ka75BqQ65SPxD-zr8goJIzoKmLLw09Uw'
'uc?export=download&id=1ka75BqQ65SPxD-zr8goJIzoKmLLw09Uw'


### Please copy paste the link starting with 'uc?export=download&confirm= ' in the next line to proceed with the implementation.

In [ ]:
!mv 'uc?export=download&confirm=bTwH&id=1ka75BqQ65SPxD-zr8goJIzoKmLLw09Uw' 'twitter_sentiment_dataset.csv'

In [ ]:
!ls

price_combined.csv  tweets_combined.csv
sample_data	    twitter_sentiment_dataset.csv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from tqdm.notebook import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch

if torch.cuda.is_available():    

    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
%matplotlib inline
sns.set(color_codes=True)

In [ ]:
df = pd.read_csv('twitter_sentiment_dataset.csv', encoding='latin-1', header = None)
df.columns=['Sentiment', 'id', 'Date', 'Query', 'User', 'Tweet']
df = df.drop(columns=['id', 'Date', 'Query', 'User'], axis=1)

In [ ]:
df.head()

In [ ]:
sns.countplot(df.Sentiment)

In [ ]:
df['Sentiment'] = df.Sentiment.replace(4,1)

In [ ]:
sns.countplot(df.Sentiment)

In [ ]:
import re

hashtags = re.compile(r"^#\S+|\s#\S+")
mentions = re.compile(r"^@\S+|\s@\S+")
urls = re.compile(r"https?://\S+")

def process_text(text):
    text = re.sub(r'http\S+', 'URL', text)
    text = hashtags.sub(' ', text)
    text = mentions.sub('AT_USER', text)
    return text.strip().lower()

In [ ]:
df['Tweet'] = df.Tweet.apply(process_text)

In [ ]:
df.head()

In [ ]:
df = df.sample(frac=1)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

In [ ]:
test.to_csv('twitter_test.csv',index=False)

In [ ]:
labels = train.Sentiment.values
sentences = train.Tweet.values

In [ ]:
from transformers import BertTokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
print(' Original: ', sentences[0])
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  fuck why is my cable broken all of the sudden
Tokenized:  ['fuck', 'why', 'is', 'my', 'cable', 'broken', 'all', 'of', 'the', 'sudden']
Token IDs:  [6616, 2339, 2003, 2026, 5830, 3714, 2035, 1997, 1996, 5573]


In [ ]:
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 64,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  fuck why is my cable broken all of the sudden
Token IDs: tensor([ 101, 6616, 2339, 2003, 2026, 5830, 3714, 2035, 1997, 1996, 5573,  102,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])


In [ ]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

1,293,487 training samples
143,721 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )
validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
def bert_base_adamw_llrd(model):
    
    opt_parameters = []
    named_parameters = list(model.named_parameters()) 

    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    init_lr = 3.5e-6 
    head_lr = 3.6e-6
    lr = init_lr
    
    params_0 = [p for n,p in named_parameters if ("pooler" in n or "regressor" in n) 
                and any(nd in n for nd in no_decay)]
    params_1 = [p for n,p in named_parameters if ("pooler" in n or "regressor" in n)
                and not any(nd in n for nd in no_decay)]
    
    head_params = {"params": params_0, "lr": head_lr, "weight_decay": 0.0}    
    opt_parameters.append(head_params)
        
    head_params = {"params": params_1, "lr": head_lr, "weight_decay": 0.01}    
    opt_parameters.append(head_params)

    for layer in range(11,-1,-1):        
        params_0 = [p for n,p in named_parameters if f"encoder.layer.{layer}." in n 
                    and any(nd in n for nd in no_decay)]
        params_1 = [p for n,p in named_parameters if f"encoder.layer.{layer}." in n 
                    and not any(nd in n for nd in no_decay)]
        
        layer_params = {"params": params_0, "lr": lr, "weight_decay": 0.0}
        opt_parameters.append(layer_params)   
                            
        layer_params = {"params": params_1, "lr": lr, "weight_decay": 0.01}
        opt_parameters.append(layer_params)       
        
        lr *= 0.9     
    
    params_0 = [p for n,p in named_parameters if "embeddings" in n 
                and any(nd in n for nd in no_decay)]
    params_1 = [p for n,p in named_parameters if "embeddings" in n
                and not any(nd in n for nd in no_decay)]
    
    embed_params = {"params": params_0, "lr": lr, "weight_decay": 0.0} 
    opt_parameters.append(embed_params)
        
    embed_params = {"params": params_1, "lr": lr, "weight_decay": 0.01} 
    opt_parameters.append(embed_params)        
    
    return AdamW(opt_parameters, lr=init_lr)

In [ ]:
optimizer = bert_base_adamw_llrd(model)

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 1
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 50,
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np

seed_val = 8

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []
total_t0 = time.time()

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():        

            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        loss = result.loss
        logits = result.logits
        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 1 ========
Training...
  Batch   500  of  80,843.    Elapsed: 0:01:02.
  Batch 1,000  of  80,843.    Elapsed: 0:02:05.
  Batch 1,500  of  80,843.    Elapsed: 0:03:07.
  Batch 2,000  of  80,843.    Elapsed: 0:04:09.
  Batch 2,500  of  80,843.    Elapsed: 0:05:11.
  Batch 3,000  of  80,843.    Elapsed: 0:06:14.
  Batch 3,500  of  80,843.    Elapsed: 0:07:16.
  Batch 4,000  of  80,843.    Elapsed: 0:08:18.
  Batch 4,500  of  80,843.    Elapsed: 0:09:21.
  Batch 5,000  of  80,843.    Elapsed: 0:10:23.
  Batch 5,500  of  80,843.    Elapsed: 0:11:25.
  Batch 6,000  of  80,843.    Elapsed: 0:12:28.
  Batch 6,500  of  80,843.    Elapsed: 0:13:30.
  Batch 7,000  of  80,843.    Elapsed: 0:14:32.
  Batch 7,500  of  80,843.    Elapsed: 0:15:34.
  Batch 8,000  of  80,843.    Elapsed: 0:16:36.
  Batch 8,500  of  80,843.    Elapsed: 0:17:39.
  Batch 9,000  of  80,843.    Elapsed: 0:18:41.
  Batch 9,500  of  80,843.    Elapsed: 0:19:43.
  Batch 10,000  of  80,843.    Elapsed: 0:20:

In [ ]:
import pandas as pd

pd.set_option('precision', 2)
df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.35,0.33,0.86,2:47:48,0:05:06


In [ ]:
df = pd.read_csv("twitter_test.csv")
df = df.dropna()

print('Number of test sentences: {:,}\n'.format(df.shape[0]))
sentences = df.Tweet.values
labels = df.Sentiment.values

input_ids = []
attention_masks = []

for sent in sentences:

    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 64,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
 
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

batch_size = 32  
prediction_data = TensorDataset(input_ids, attention_masks,labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 159,704



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

model.eval()

predictions ,predictions_probs, true_labels = [], [], []
for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask,b_labels = batch
  with torch.no_grad():
      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits
  probabilities = F.softmax(logits, dim=1)
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  probabilities = probabilities.detach().cpu().numpy()
  predictions.append(logits)
  true_labels.append(label_ids)
  predictions_probs.append(probabilities)

print('    DONE.')

Predicting labels for 159,704 test sentences...
    DONE.


In [ ]:
flat_predictions = np.concatenate(predictions, axis=0)
flat_predictions_probs = np.concatenate(predictions_probs, axis=0)

flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = np.concatenate(true_labels, axis=0)

avg_test_accuracy = np.sum(flat_predictions == flat_true_labels) / len(flat_true_labels)

print("  Accuracy: {0:.2f}".format(avg_test_accuracy))
print(flat_predictions[10],flat_true_labels[10],flat_predictions_probs[10])

  Accuracy: 0.86
0 0 [0.99167085 0.00832917]


In [ ]:
len(list(flat_predictions))

1000

In [ ]:
outputs = list(flat_predictions)

In [ ]:
df_output = pd.DataFrame(list(zip(list(flat_predictions), list(flat_true_labels),list(flat_predictions_probs))),columns = ['predictions','true_labels','pred_probs'])

In [ ]:
df_output[df_output['predictions']==0]

,predictions,true_labels,pred_probs
2,0,0,"[0.9886778, 0.01132213]"
4,0,0,"[0.9790299, 0.020970104]"
7,0,0,"[0.9965725, 0.0034275427]"
8,0,0,"[0.93998504, 0.060014993]"
9,0,0,"[0.98344064, 0.01655942]"
...,...,...,...
159694,0,0,"[0.9955395, 0.0044605345]"
159695,0,1,"[0.5669716, 0.43302837]"
159697,0,1,"[0.8787022, 0.12129773]"
159698,0,0,"[0.8672617, 0.13273834]"


In [ ]:
df_test_output = pd.merge(df,df_output,left_index=True,right_index=True,how='inner')

In [ ]:
df_test_output.head()

,Sentiment,Tweet,predictions,true_labels,pred_probs
0,1,eating a banana - i also just fit into a size ...,1,1,"[0.026936928, 0.97306305]"
1,1,"awesome! we had a lot of family time, too. tal...",1,1,"[0.008765864, 0.9912341]"
2,0,eep. just had a waking nightmare. i don't know...,0,0,"[0.9886778, 0.01132213]"
3,1,i'm thinking maybe i should raid the fridge n ...,1,1,"[0.10359162, 0.89640844]"
4,0,i miss you,0,0,"[0.9790299, 0.020970104]"


In [ ]:
df_test_output.to_csv('df_test_output.csv',index=False)

In [ ]:
df = pd.read_csv("tweets_combined.csv",encoding='latin-1',lineterminator='\n')

In [ ]:
df.head()

,stock_symbol,tweet_id,time,tweet
0,AAPL,583947352286875648,Fri Apr 03 11:01:12 +0000 2015,RT NewsSDRL: RT NewsSDRL: #toptickertweets $AA...
1,AAPL,583948769961648128,Fri Apr 03 11:06:50 +0000 2015,RT NewsIACI: RT NewsIACI: RT NewsSDRL: #toptic...
2,AAPL,583950963590897665,Fri Apr 03 11:15:33 +0000 2015,10 things in tech you need to know today $GOOG...
3,AAPL,583952666491019264,Fri Apr 03 11:22:19 +0000 2015,RT NewsSDRL: RT NewsSDRL: #toptickertweets $AA...
4,AAPL,583963546490404864,Fri Apr 03 12:05:33 +0000 2015,RT NewsIACI: RT NewsIACI: RT NewsAAPL: #toptic...


In [ ]:
import re

hashtags = re.compile(r"^#\S+|\s#\S+")
mentions = re.compile(r"^@\S+|\s@\S+")
urls = re.compile(r"https?://\S+")

def process_text(text):
    text = re.sub(r'http\S+', '', text)
    text = hashtags.sub(' ', text)
    text = mentions.sub(' ', text)
    return text.strip().lower()

In [ ]:
df['tweet'] = df.tweet.apply(process_text)

In [ ]:
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

sentences = df.tweet.values

input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 64,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
batch_size = 32  
prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 119,852



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

model.eval()
predictions ,predictions_probs = [], []

for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask = batch

  with torch.no_grad():
      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits
  probabilities = F.softmax(logits, dim=1)
  logits = logits.detach().cpu().numpy()
  probabilities = probabilities.detach().cpu().numpy()
  predictions.append(logits)
  predictions_probs.append(probabilities)

print('    DONE.')

Predicting labels for 119,852 test sentences...
    DONE.


In [ ]:
flat_predictions = np.concatenate(predictions, axis=0)
flat_predictions_probs = np.concatenate(predictions_probs, axis=0)
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
print(flat_predictions[10],flat_predictions_probs[10])

1 [0.08424371 0.9157563 ]


In [ ]:
df_output = pd.DataFrame(list(zip(list(flat_predictions),list(flat_predictions_probs))),columns = ['predictions','pred_probs'])

In [ ]:
df_test_output = pd.merge(df,df_output,left_index=True,right_index=True,how='inner')

In [ ]:
df_test_output.head()

,stock_symbol,tweet_id,time,tweet,predictions,pred_probs
0,AAPL,583947352286875648,Fri Apr 03 11:01:12 +0000 2015,rt newssdrl: rt newssdrl: $aapl $spy $tza $ts...,1,"[0.40755036, 0.5924496]"
1,AAPL,583948769961648128,Fri Apr 03 11:06:50 +0000 2015,rt newsiaci: rt newsiaci: rt newssdrl: $aapl ...,1,"[0.39781833, 0.6021817]"
2,AAPL,583950963590897665,Fri Apr 03 11:15:33 +0000 2015,10 things in tech you need to know today $goog...,1,"[0.09492045, 0.9050796]"
3,AAPL,583952666491019264,Fri Apr 03 11:22:19 +0000 2015,rt newssdrl: rt newssdrl: $aapl $spy $tza $ts...,1,"[0.40755036, 0.5924496]"
4,AAPL,583963546490404864,Fri Apr 03 12:05:33 +0000 2015,rt newsiaci: rt newsiaci: rt newsaapl: $aapl ...,1,"[0.424042, 0.575958]"


In [ ]:
df_test_output.to_csv('tweets_combined_sentiment_output.csv',index=False)